In [1]:
import torch

In [2]:
# Import uncertainty-aware 2D convolution (UAConv2d)
from ualib import UAConv2d

# Define random input: x_mean (input means tensor) and x_var (input variances tensor);
# Note that x_var's elements should be positive (as variances)
batch_size=16
inp_channels=3
inp_dim=224
x_mean = torch.randn(batch_size, inp_channels, inp_dim, inp_dim)
x_var = torch.rand(batch_size, inp_channels, inp_dim, inp_dim)

# Define an UAConv2D layer
uaconv2d = UAConv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=0)

# Forward input (pair of means and variances tensors) through the UAConv2D layer; 
# the result is again a pair of means and variances tensors. 
[y_mean, y_var] = uaconv2d([x_mean, x_var])

In [3]:
# Import uncertainty-aware ReLU (UAReLU)
from ualib import UAReLU

# Define an UAReLU layer
uarelu = UAReLU()

# Apply UAReLU to the output of the previous UAConv2d
[y_mean, y_var] = uarelu([y_mean, y_var]) 

In [4]:
# Import uncertainty-aware average pooling (UAAvgPool2d)
from ualib import UAAvgPool2d

# Define an UAAvgPool2d layer
uaavgpool2d = UAAvgPool2d(kernel_size=2, stride=2)

# Apply UAAvgPool2d on the output of the previous UAReLU
[y_mean, y_var] = uaavgpool2d([y_mean, y_var]) 

In [5]:
# Flatten means and variances tensors
y_mean = y_mean.view(y_mean.size(0), -1)
y_var = y_var.view(y_var.size(0), -1)

In [6]:
# Import uncertainty-aware linear (fully-connected) layer (UALinear)
from ualib import UALinear

# Define an UALinear layer
ualinear = UALinear(in_features=y_mean.size(1), out_features=1)

# Apply UALinear to the (flattened) output of the previous UAAvgPool2d
[y_mean, y_var] = ualinear([y_mean, y_var])

In [7]:
# Import uncertainty-aware sigmoid function (UASigmoid)
from ualib import UASigmoid

# Define an UASigmoid layer
uasigmoid = UASigmoid()

# Apply UASigmoid to the previous UALinear output
[y_mean, y_var] = uasigmoid([y_mean, y_var])

In [8]:
# Import uncertainty-aware binary cross entropy loss (UABCELoss)
from ualib import UABCELoss

# Define UABCELoss function
uabce_loss = UABCELoss()

# Define random targets
targets = torch.empty(batch_size).random_(2)

# Compute UABCELoss
loss = uabce_loss(y_mean, y_var, targets)